# Test random number generator

In [1]:
from system.noise import NormalNoiseGenerator

In [2]:
ng = NormalNoiseGenerator(mean=0.0, std_dev=1.0, dimension=3, seed=42)

S1 = next(ng)
S2 = next(ng)
S3 = next(ng)

TypeError: object of type 'float' has no len()

In [3]:
ng1 = NormalNoiseGenerator(mean=0.0, std_dev=1.0, dimension=3, seed=42)
ng2 = NormalNoiseGenerator(mean=0.0, std_dev=1.0, dimension=3, seed=91)

assert S1 == next(ng1), f"(S1) {S1} != {next(ng1)}"
_1 = next(ng2)
assert S2 == next(ng1), f"(S2) {S2} != {next(ng1)}"
_2 = next(ng2)
_3 = next(ng2)
_4 = next(ng2)
assert S3 == next(ng1), f"(S3) {S3} != {next(ng1)}"

# Test adding monomials to equation

In [4]:
from system.equation import Equation, Monomial

initial_monomials = [
    Monomial(1, ["X", "Y", "Z"], [1,2,3]),
    Monomial(2, ["X", "Y", "Z"], [0,0,2]),
]
eq = Equation(initial_monomials)
print(eq) # (A * X * Y**2 * Z**3) + (B * Z**2)

(X * Y**2 * Z**3) + (2 * Z**2)


In [5]:
eq.add_monomial(Monomial("C", ["X", "Y", "Z"], [0,0,0]))
print(eq) # (A * X * Y**2 * Z**3) + (B * Z**2) + (C)

TypeError: type str doesn't define __round__ method

In [6]:
eq.add_monomial(Monomial("D", ["X", "Y", "Z"], [1,2,3]))
print(eq) # ((A + D) * X * Y**2 * Z**3) + (B * Z**2) + (C)

(AD * X * Y**2 * Z**3) + (B * Z**2) + (C * )


In [7]:
eq.add_monomial(Monomial("-E", ["X", "Y", "Z"], [1,2,3]))
eq.add_monomial(Monomial("-E", ["X", "Y", "Z"], [0,0,0]))
print(eq) # ((A + D + -E) * X * Y**2 * Z**3) + (B * Z**2) + (C + -E)

(AD-E * X * Y**2 * Z**3) + (B * Z**2) + (C-E * )


In [8]:
eq.add_monomial(Monomial("-M", ["X", "Y", "Z"], [7,0,0]))
print(eq)

(AD-E * X * Y**2 * Z**3) + (B * Z**2) + (C-E * ) + (-M * X**7)


## Construction test (reverse)

In [6]:
from system.equation import Equation

_eq_str = "(A) + (B * Z**2) + (C) + (D * X * Y**2 * Z**3) + (-E * X * Y**2 * Z**3) + (-E) + (-M * X**7)"
_eq_str2 = "A + B * Z**2 + C + D * X * Y**2 * Z**3 + -E * X * Y**2 * Z**3 -E  -M * X**7"

eq = Equation.extract_equation_from_string(_eq_str)
eq2 = Equation.extract_equation_from_string(_eq_str2)
print("[1]", eq)
print("[2]", eq2)
assert str(eq).replace(" ","").replace("-1*","-") == _eq_str.replace(" ",""), f"{str(eq)} != {_eq_str}"
assert str(eq2) == str(eq), f"{str(eq2)} != {eq}"

[1] (A) + (B * Z**2) + (C) + (D * X * Y**2 * Z**3) + (-1 * E * X * Y**2 * Z**3) + (-1 * E) + (-1 * M * X**7)
[2] (A) + (B * Z**2) + (C) + (D * X * Y**2 * Z**3) + (-1 * E * X * Y**2 * Z**3) + (-1 * E) + (-1 * M * X**7)


In [7]:
for m in eq.monomials:
    print(f"{int(m.coefficient):<2} >> ", end="")
    for v,p in zip(m.variable_generators, m.power):
        print(f"{v:>2}^{p:<2}", end=" ")
    print(f" >> {m}")
print()
print(eq)

1  >>  A^1   >> A
1  >>  B^1   Z^2   >> B * Z**2
1  >>  C^1   >> C
1  >>  D^1   X^1   Y^2   Z^3   >> D * X * Y**2 * Z**3
-1 >>  E^1   X^1   Y^2   Z^3   >> -1 * E * X * Y**2 * Z**3
-1 >>  E^1   >> -1 * E
-1 >>  M^1   X^7   >> -1 * M * X**7

(A) + (B * Z**2) + (C) + (D * X * Y**2 * Z**3) + (-1 * E * X * Y**2 * Z**3) + (-1 * E) + (-1 * M * X**7)


# Test Equation generation in Action

In [8]:
from system.action import SystemControlPolicy
from system.space import Space

In [9]:
scp = SystemControlPolicy(
    action_space=Space(2, ""),
    state_dimension=2,
    maximal_degree=3,
)

In [10]:
print(len(scp.transitions))
for eq in scp.transitions:
    print(len(eq.monomials))

2
10
10


In [11]:
print(scp.transitions[0])

(P1_1) + (P1_2 * S2) + (P1_3 * S2**2) + (P1_4 * S2**3) + (P1_5 * S1) + (P1_6 * S1 * S2) + (P1_7 * S1 * S2**2) + (P1_8 * S1**2) + (P1_9 * S1**2 * S2) + (P1_10 * S1**3)


# Test the parser

In [15]:
from system.toolIO import Parser

In [16]:
parser = Parser("./kaushik_sample/config.json")

In [17]:
tool_input = parser.parse()
print(tool_input)

ToolInput(state_space=Space(dimension=2, inequalities='0 <= S1 <= 2 and -2 <= S2 <= 0'), action_policy=SystemControlPolicy(action_space=Space(dimension=2, inequalities='0 <= A1 <= 1 and -1 <= A2 <= 1'), state_dimension=2, maximal_degree=2, transitions=[Equation(monomials=[Monomial(coefficient=1, variable_generators=('P1_1', 'S1', 'S2'), power=(1, 0, 0)), Monomial(coefficient=1, variable_generators=('P1_2', 'S1', 'S2'), power=(1, 0, 1)), Monomial(coefficient=1, variable_generators=('P1_3', 'S1', 'S2'), power=(1, 0, 2)), Monomial(coefficient=1, variable_generators=('P1_4', 'S1', 'S2'), power=(1, 1, 0)), Monomial(coefficient=1, variable_generators=('P1_5', 'S1', 'S2'), power=(1, 1, 1)), Monomial(coefficient=1, variable_generators=('P1_6', 'S1', 'S2'), power=(1, 2, 0))]), Equation(monomials=[Monomial(coefficient=1, variable_generators=('P2_1', 'S1', 'S2'), power=(1, 0, 0)), Monomial(coefficient=1, variable_generators=('P2_2', 'S1', 'S2'), power=(1, 0, 1)), Monomial(coefficient=1, variable_

In [18]:
parser2 = Parser(
    "./input_sample_2/input1.json", 
    "./input_sample_2/input2.json", 
    "./input_sample_2/input3.json"
)

In [19]:
tool_input2 = parser.parse()
print(tool_input)

ToolInput(state_space=Space(dimension=2, inequalities='0 <= S1 <= 2 and -2 <= S2 <= 0'), action_policy=SystemControlPolicy(action_space=Space(dimension=2, inequalities='0 <= A1 <= 1 and -1 <= A2 <= 1'), state_dimension=2, maximal_degree=2, transitions=[Equation(monomials=[Monomial(coefficient=1, variable_generators=('P1_1', 'S1', 'S2'), power=(1, 0, 0)), Monomial(coefficient=1, variable_generators=('P1_2', 'S1', 'S2'), power=(1, 0, 1)), Monomial(coefficient=1, variable_generators=('P1_3', 'S1', 'S2'), power=(1, 0, 2)), Monomial(coefficient=1, variable_generators=('P1_4', 'S1', 'S2'), power=(1, 1, 0)), Monomial(coefficient=1, variable_generators=('P1_5', 'S1', 'S2'), power=(1, 1, 1)), Monomial(coefficient=1, variable_generators=('P1_6', 'S1', 'S2'), power=(1, 2, 0))]), Equation(monomials=[Monomial(coefficient=1, variable_generators=('P2_1', 'S1', 'S2'), power=(1, 0, 0)), Monomial(coefficient=1, variable_generators=('P2_2', 'S1', 'S2'), power=(1, 0, 1)), Monomial(coefficient=1, variable_

# Test System Dynamics

In [12]:
from system.dynamics import SystemDynamics
from system.equation import Equation

sd = SystemDynamics(
    state_dimension=5,
    action_dimension=3,
    disturbance_dimension=1,
    system_transformations=[
        Equation.extract_equation_from_string("S1 + A1 * S2 + D1"),
        Equation.extract_equation_from_string("73*S1 + S2 + S2"),
        Equation.extract_equation_from_string("A1 * D1 + (-S4)"),
        Equation.extract_equation_from_string("D1 + S1 + A2*12*19"),
        Equation.extract_equation_from_string("S1 + S2 + D1"),
    ],
)

In [13]:
print(sd([],[],[]))

TypeError: 'list' object is not callable

In [14]:
sd(state=[1,2,3,4,5], action=[1,-5,3], noise=[1])

TypeError: 'list' object is not callable

In [15]:
sd(state=[1,2,3,4,5], action=[1,-5,3], noise=[1], evaluate=True)

TypeError: 'list' object is not callable

In [16]:
sd(
    state=[1,2,3,"A9 + B0 ** 10",5], 
    action=[1,-5,3], 
    noise=[1], 
    evaluate=False
)

TypeError: 'list' object is not callable

# Test Space Inequality extraction

In [17]:
from system.space import Space

In [19]:
_s1 = Space(dimension=3, inequalities="21 < X1 < 21 and 92 > X1+X2*3 >= 11")
print(_s1.get_space_inequalities())

[Inequality(left_equation=Equation(monomials=[Monomial(coefficient=21, variable_generators=[], power=[]), Monomial(coefficient=-1, variable_generators=('X1',), power=(1,))]), inequality_type=<EquationConditionType.GREATER_THAN_OR_EQUAL: '>='>, right_equation=Equation(monomials=[])), Inequality(left_equation=Equation(monomials=[Monomial(coefficient=92, variable_generators=[], power=[]), Monomial(coefficient=-1, variable_generators=('X1',), power=(1,)), Monomial(coefficient=-3, variable_generators=('X2',), power=(1,))]), inequality_type=<EquationConditionType.GREATER_THAN_OR_EQUAL: '>='>, right_equation=Equation(monomials=[])), Inequality(left_equation=Equation(monomials=[Monomial(coefficient=1, variable_generators=('X1',), power=(1,)), Monomial(coefficient=-21, variable_generators=[], power=[])]), inequality_type=<EquationConditionType.GREATER_THAN_OR_EQUAL: '>='>, right_equation=Equation(monomials=[])), Inequality(left_equation=Equation(monomials=[Monomial(coefficient=1, variable_gener